In [1]:
import requests
import os
import json
import pandas as pd

from dotenv import load_dotenv

In [2]:
 # Load environment variables from .env file
load_dotenv()

# Get the API key
api_key = os.getenv('NOAA_CLIMATE_DATA')
token = api_key

In [3]:
def get_weather_data(token, location_id, start_date, end_date):
    url = "https://www.ncdc.noaa.gov/cdo-web/api/v2/data"
    headers = {
        'token': token
    }
    params = {
        'datasetid': 'GSOM',  # Global Historical Climatology Network - Daily
        'locationid': location_id,  # Location ID for the city
        'startdate': start_date,
        'enddate': end_date,
        'datatypeid': data_type,
        'units': 'standard',  # Use 'standard' for Fahrenheit, inches, etc.
        'limit': 1000  # Adjust as needed
    }

    response = requests.get(url, headers=headers, params=params)
    print(response)
    
    if response.status_code == 200:
        return response.json()
    else:
        return response.status_code, response.text



In [4]:
#Load country codes to data frame
codes_path = 'Resources/ghcnd_country_codes.csv'
code_country_df = pd.read_csv(codes_path, sep=',', header=0)
location_ids = code_country_df['Code'].tolist()
print(code_country_df.head())

#Date types to get
data_type = ["DP10", "DP1X", "DT32", "DX70", "DX90", "PRCP", "RHAV", "TAVG", 'TMAX']

  Code
0   AC
1   AE
2   AF
3   AG
4   AJ


In [5]:
#Start date list
start_date = [
    '1995-01-01',
    '1995-07-01',
    '1996-01-01',
    '1996-07-01',
    '1997-01-01',
    '1997-07-01',
    '1998-01-01',
    '1998-07-01',
    '1999-01-01',
    '1999-07-01',
    '2000-01-01',
    '2000-07-01',
    '2001-01-01',
    '2001-07-01',
    '2002-01-01',
    '2002-07-01',
    '2003-01-01',
    '2003-07-01',
    '2004-01-01',
    '2004-07-01',
    '2005-01-01',
    '2005-07-01',
    '2006-01-01',
    '2006-07-01',
    '2007-01-01',
    '2007-07-01',
    '2008-01-01',
    '2008-07-01',
    '2009-01-01',
    '2009-07-01',
    '2010-01-01',
    '2010-07-01',
    '2011-01-01',
    '2011-07-01',
    '2012-01-01',
    '2012-07-01',
    '2013-01-01',
    '2013-07-01',
    '2014-01-01',
    '2014-07-01',
    '2015-01-01',
    '2015-07-01',
    '2016-01-01',
    '2016-07-01',
    '2017-01-01',
    '2017-07-01',
    '2018-01-01',
    '2018-07-01',
    '2019-01-01',
    '2019-07-01',
    '2020-01-01',
    '2020-07-01',
    '2021-01-01',
    '2021-07-01',
    '2022-01-01',
    '2022-07-01',
    '2023-01-01',
    '2023-07-01'
]


In [6]:
#End data list
end_date = [
    '1995-01-01',
    '1995-07-01',
    '1996-01-01',
    '1996-07-01',
    '1997-01-01',
    '1997-07-01',
    '1998-01-01',
    '1998-07-01',
    '1999-01-01',
    '1999-07-01',
    '2000-01-01',
    '2000-07-01',
    '2001-01-01',
    '2001-07-01',
    '2002-01-01',
    '2002-07-01',
    '2003-01-01',
    '2003-07-01',
    '2004-01-01',
    '2004-07-01',
    '2005-01-01',
    '2005-07-01',
    '2006-01-01',
    '2006-07-01',
    '2007-01-01',
    '2007-07-01',
    '2008-01-01',
    '2008-07-01',
    '2009-01-01',
    '2009-07-01',
    '2010-01-01',
    '2010-07-01',
    '2011-01-01',
    '2011-07-01',
    '2012-01-01',
    '2012-07-01',
    '2013-01-01',
    '2013-07-01',
    '2014-01-01',
    '2014-07-01',
    '2015-01-01',
    '2015-07-01',
    '2016-01-01',
    '2016-07-01',
    '2017-01-01',
    '2017-07-01',
    '2018-01-01',
    '2018-07-01',
    '2019-01-01',
    '2019-07-01',
    '2020-01-01',
    '2020-07-01',
    '2021-01-01',
    '2021-07-01',
    '2022-01-01',
    '2022-07-01',
    '2023-01-01',
    '2023-07-01'
]


In [7]:
# Create an empty DataFrame to store results
final_df = pd.DataFrame()

# Loop through each country and date range
for location_id in location_ids:
    for start_date, end_date in zip(start_date, end_date):
        weather_data = get_weather_data(token, location_id, start_date, end_date)
        if weather_data:
            # Convert the JSON response to a DataFrame
            weather_df = pd.json_normalize(weather_data['results'])
            # Append to the final DataFrame
            final_df = pd.concat([final_df, weather_df], ignore_index=True)

# Export the final DataFrame to a CSV file
final_df.to_csv('weather_data.csv', index=False)

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [503]>


TypeError: tuple indices must be integers or slices, not str